In [1]:
import project_diff as project_diff


In [2]:
oldprojectfile = 'sample_files/ISL69269-0 0x60a.txt'
newprojectfile = 'sample_files/ISL69269-0 0x60b.txt'
outputfilename = 'diff_file_ab.txt'

In [3]:

diff = project_diff.file_delta(oldprojectfile,newprojectfile,outputfilename)

NameError: name 'command_dict' is not defined

In [4]:
diff.delta

,line,name,value,address
1008,136,rGUI5,A1A17E61,003F
228,228,PAGE,00,0
1009,229,droop,00E6,28
1010,913,rampCompBulkA0,00002407,EA2C


In [5]:
diff.write_outputfile()